# Loading Model Requirements

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib notebook

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
import random
import math


import tensorflow as tf
import tensorflow_probability as tfp

from IPython import display
from sklearn import preprocessing
from pickle import dump, load

from matplotlib.ticker import FormatStrFormatter
from IPython.display import SVG

tf.random.set_seed(1234)

tfd = tfp.distributions
tfb = tfp.bijectors
tfk = tf.keras

# Loading Data and Preprocessing

In [0]:
import math
def lonlat2meters(lon, lat):
    semimajoraxis = 6378137.0
    east = lon * 0.017453292519943295
    north = lat * 0.017453292519943295
    t = math.sin(north)
    return semimajoraxis * east, 3189068.5 * math.log((1 + t) / (1 - t))

def meters2lonlat(x, y):
    semimajoraxis = 6378137.0
    lon = x / semimajoraxis / 0.017453292519943295
    t = math.exp(y / 3189068.5)
    lat = math.asin((t - 1) / (t + 1)) / 0.017453292519943295
    return lon, lat

In [0]:
dataset = np.genfromtxt('../processed_nyc_train.csv',delimiter=',', skip_header=1)
dataset = dataset[~np.isnan(dataset).any(axis=1)]

def format_data(dataset, pick_up_scaler=None, drop_off_scaler = None ,save_scaler=True):
    
    pick_up_c, drop_off_c, num_passenger, travel_duration = np.split(dataset, [2, 4, 5], axis = 1)
    
    # Handling of the coordinates
    for i, c in enumerate(pick_up_c):
        lon = pick_up_c[i][0]
        lat = pick_up_c[i][1]
        x, y = lonlat2meters(lon, lat)
        pick_up_c[i][0] = x
        pick_up_c[i][1] = y
    
    if pick_up_scaler is None:
        pick_up_scaler = preprocessing.StandardScaler()
        pick_up_scaler = pick_up_scaler.fit(pick_up_c)
    
    pick_up_c = pick_up_scaler.transform(pick_up_c)
    
    for i, c in enumerate(drop_off_c):
        lon = drop_off_c[i][0]
        lat = drop_off_c[i][1]
        x, y = lonlat2meters(lon, lat)
        drop_off_c[i][0] = x
        drop_off_c[i][1] = y
    
    if drop_off_scaler is None:
        drop_off_scaler = preprocessing.StandardScaler()
        drop_off_scaler = drop_off_scaler.fit(drop_off_c)
    
    drop_off_c = drop_off_scaler.transform(drop_off_c)
    
    
    if save_scaler:
        dump(pick_up_scaler, open('pick_up_scaler.pkl', 'wb'))
        dump(drop_off_scaler, open('drop_off_scaler.pkl', 'wb'))

    final = np.concatenate([pick_up_c, drop_off_c, num_passenger, travel_duration], axis = 1)
    return final

dataset = format_data(dataset)

# Model Definition

In [0]:
import time

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.layers import Layer, Dense, BatchNormalization, ReLU, Conv2D, Reshape
from tensorflow.keras import Model

tfd = tfp.distributions
tfb = tfp.bijectors
tfk = tf.keras

tf.keras.backend.set_floatx('float32')

print('tensorflow: ', tf.__version__)
print('tensorflow-probability: ', tfp.__version__)

from enum import Enum

class Case(Enum):
    sampling = 1
    density_estimation = 2

class Made(tfk.layers.Layer):
    """
    Implementation of a Masked Autoencoder for Distribution Estimation (MADE) [Germain et al. (2015)].
    The existing TensorFlow bijector "AutoregressiveNetwork" is used. The output is reshaped to output one shift vector
    and one log_scale vector.

    :param params: Python integer specifying the number of parameters to output per input.
    :param event_shape: Python list-like of positive integers (or a single int), specifying the shape of the input to this layer, which is also the event_shape of the distribution parameterized by this layer. Currently only rank-1 shapes are supported. That is, event_shape must be a single integer. If not specified, the event shape is inferred when this layer is first called or built.
    :param hidden_units: Python list-like of non-negative integers, specifying the number of units in each hidden layer.
    :param activation: An activation function. See tf.keras.layers.Dense. Default: None.
    :param use_bias: Whether or not the dense layers constructed in this layer should have a bias term. See tf.keras.layers.Dense. Default: True.
    :param kernel_regularizer: Regularizer function applied to the Dense kernel weight matrices. Default: None.
    :param bias_regularizer: Regularizer function applied to the Dense bias weight vectors. Default: None.
    """

    def __init__(self, params, event_shape=None, hidden_units=None, activation=None, use_bias=True,
                 kernel_regularizer=None, bias_regularizer=None):

        super(Made, self).__init__()

        self.params = params
        self.event_shape = event_shape
        self.hidden_units = hidden_units
        self.activation = activation
        self.use_bias = use_bias
        self.kernel_regularizer = kernel_regularizer
        self.bias_regularizer = bias_regularizer

        self.network = tfb.AutoregressiveNetwork(params=params, event_shape=event_shape, hidden_units=hidden_units,
                                                 activation=activation, use_bias=use_bias, kernel_regularizer=kernel_regularizer, 
                                                 bias_regularizer=bias_regularizer)

    def call(self, x):
        shift, log_scale = tf.unstack(self.network(x), num=2, axis=-1)

        return shift, tf.math.tanh(log_scale)

tensorflow:  2.2.0-rc2
tensorflow-probability:  0.9.0


In [0]:
hidden_shape = [100, 100]  # hidden shape for MADE network of MAF
layers = 12  # number of layers of the flow
event_shape=[6]

base_dist = tfd.Normal(loc=0.0, scale=1.0)  # specify base distribution

bijectors = []
for i in range(0, layers):
    bijectors.append(tfb.MaskedAutoregressiveFlow(shift_and_log_scale_fn = Made(params=2, event_shape=event_shape, hidden_units=hidden_shape, activation="relu")))
    bijectors.append(tfb.Permute(permutation=[5,4,3,2,1,0]))  # data permutation after layers of MAF
    
bijector = tfb.Chain(bijectors=list(reversed(bijectors)), name='chain_of_maf')

maf = tfd.TransformedDistribution(
    distribution=base_dist,
    bijector=bijector,
    event_shape=event_shape
)

# initialize flow
samples = maf.sample()

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


# Model Training Parameters

In [0]:
base_lr = 1e-3
end_lr = 1e-4
max_epochs = int(5e3)  # maximum number of epochs of the training
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(base_lr, max_epochs, end_lr, power=0.5)

In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn) 

x_ = tf.keras.layers.Input(shape=event_shape, dtype=tf.float32)
log_prob_ = maf.log_prob(x_)
model = tf.keras.Model(x_, log_prob_)

In [0]:
model.compile(optimizer=opt,
              loss=lambda _, log_prob: -log_prob)

# Setting Up Checkpoints ( Will Save Model After Every Epoch)

In [0]:
batch_size = 100
weight_file = './checkpoint/cp.h5'

In [0]:
for i in range(50): # change number of epochs accordingly
  model.load_weights(weight_file)
  model.fit(x=dataset,
            y=np.zeros((dataset.shape[0], 0), dtype=np.float32),
            batch_size=batch_size,
            epochs=1,
            steps_per_epoch=dataset.shape[0] // batch_size,
            shuffle=True,
            verbose=True)
  model.save_weights(weight_file)

 7473/14586 [==============>...............] - ETA: 2:33 - loss: 39.6500

KeyboardInterrupt: ignored

In [0]:
model.save_weights("MAF_12_100_100.h5")

# Generating Samples

In [0]:
output = maf.sample(100000).numpy()

In [0]:
def reconstruct(predictions):
    
    # split the output first
    pick_up_c, drop_off_c, num_passenger, travel_duration = np.split(dataset, [2, 4, 5], axis = 1)
    
    # recover scaler
    pick_up_scaler = load(open('pick_up_scaler.pkl', 'rb'))
    drop_off_scaler = load(open('drop_off_scaler.pkl', 'rb'))
    pick_up_c = pick_up_scaler.inverse_transform(pick_up_c)
    drop_off_c = drop_off_scaler.inverse_transform(drop_off_c)

    for i, c in enumerate(pick_up_c):
      x = pick_up_c[i][0]
      y = pick_up_c[i][1]
      lon, lat = meters2lonlat(x, y)
      pick_up_c[i][0] = lon
      pick_up_c[i][1] = lat
    
    for i, c in enumerate(pick_up_c):
      x = drop_off_c[i][0]
      y = drop_off_c[i][1]
      lon, lat = meters2lonlat(x, y)
      drop_off_c[i][0] = lon
      drop_off_c[i][1] = lat
    
    return np.concatenate([pick_up_c, drop_off_c, num_passenger, travel_duration], axis = 1)

In [0]:
samples = reconstruct(output)

# Saving Generated Samples

In [0]:
file_name = '100000_samples_MAF_12_100_100' + '.csv'
np.savetxt(file_name, samples, delimiter = ',', header='pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, passenger_count, trip_duration' )